In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# debug cuda
#import os
#os.environ['NUMBA_ENABLE_CUDASIM'] = '1'

import numba
from numba import SmartArray, cuda, float32, int8, int32
from numba.cuda.random import create_xoroshiro128p_states as make_rng_states, xoroshiro128p_uniform_float32 as get_rng
import numpy as np
from IPython import display
from scipy.misc import imshow
from functools import partial
import math
from miniutils import progbar

In [ ]:
from fl4mes import variations as v, transforms as t, runner

In [ ]:
v.plot_variation(v.swirl())

In [ ]:
np.random.seed(2)
t.plot_transform(t.Transform(), xrs=101, yrs=101)

In [ ]:
np.random.seed(0)
transforms = [t.Transform() for _ in range(10)]
transition_matrix = np.random.normal(size=(10,10))
transition_matrix /= np.sum(transition_matrix, axis=1, keepdims=True)
kernel, get_img = runner.make_kernel(10, 10000, transforms, transition_matrix, ((-5,5),(-5,5)), (128, 128), min_step=20)

plt.figure()
for i in progbar(100):
    kernel(1000)
    r, g, b, a = get_img()
    #print(np.sum(a))
    im = np.stack([r,g,b], axis=-1)
    #print(im.shape)
    rough_img = np.log1p(im)
    rough_img /= rough_img.max()
    plt.imshow(rough_img, interpolation='nearest')
    plt.show()
    plt.gcf().canvas.draw()

In [ ]:
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np

@cuda.jit
def compute_pi(rng_states, iterations, out):
    """Find the maximum value in values and store in result[0]"""
    thread_id = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[thread_id] = 4.0 * inside / iterations

threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)

compute_pi[blocks, threads_per_block](rng_states, 10000, out)
print('pi:', out.mean())